In [ ]:
# This part of the code is used for msa_generation
# An scPDB.tar.gz file is already present in the repo and it contains sequence.fasta and downloaded.pdb files for all the scPDB data points
# Let us extract that into data/scPDB
# Most of the code is taken from Deepmsa https://zhanglab.ccmb.med.umich.edu/DeepMSA/ with some small changes
!tar xvzf scPDB.tar.gz -C ../data/
# Download uniref50.fasta and uniclust30_2017_10 into data folder
!aria2c -c -x 8 -s 8 -d "../data/" ftp://ftp.uniprot.org/pub/databases/uniprot/uniref/uniref50/uniref50.fasta.gz
!aria2c -c -x 8 -s 8 -d "../data/" http://wwwuser.gwdg.de/~compbiol/uniclust/2017_10/uniclust30_2017_10_hhsuite.tar.gz
!tar xvzf ./data/uniref50.fasta.gz -C ./data/
!tar xvzf ./data/uniclust30_2017_10_hhsuite.tar.gz -C ./data/

In [3]:
# Some important constants
# ENSURE THAT YOU RUN THIS CODE CELL FIRST
import os
from utils import *

data_dir = os.path.abspath("../data")
dataset_dir = os.path.join(data_dir, "scPDB")
raw_dir = os.path.join(dataset_dir, "raw")
splits_dir = os.path.join(raw_dir, "splits")


In [ ]:
# We need to generate MSAs for the protein sequences in the dataset
# For that, we need to split the sequence.fasta file into respective chain.fasta files
# Also, we need to remove the fasta files of DNA/RNA seqeuences

for file in sorted(os.listdir(raw_dir)):
    file = file.strip()
    pre = os.path.join(raw_dir, file)

    # Read SEQRES entries in PDB file to determine whether a chain
    # has a protein sequence or not
    pdb_file = os.path.join(pre, "downloaded.pdb")
    do_not_include = set()
    with open(pdb_file, "r") as f:
        line = f.readline()
        while line[:6] != "SEQRES":
            line = f.readline()
        while line[:6] == "SEQRES":
            chain_id = line[11]
            residue = line[19:22]
            # Generally DNA/RNA have 1 or 2-letter codes
            if " " in residue:
                do_not_include.add(chain_id)
            line = f.readline()

    fasta = os.path.join(pre, "sequence.fasta")
    create_chains_from_sequence_fasta(pre, fasta, do_not_include)


In [ ]:
# In case you want to delete the generated fasta files from the above cell, use this
# for file in sorted(os.listdir(raw_dir)):
#     for fasta in glob(os.path.join(raw_dir, file.strip(), "?.fasta")):
#         os.remove(fasta)


In [ ]:
# The fasta files generated will have a lot of common sequences
# To speed up MSA generation, let us create a unique file that has common sequences
# Then we can generate the MSAs for only the first chain in every line
make_unique_file(raw_dir, splits_dir)

# Let us split the MSAs into a 100 files so that they can all be run parallely
split_unique_file(splits_dir, 100)


In [ ]:
# Ensure that boost regex library has been installed
# MSAs take a lot of time to generate. Assuming a SLURM Workload Manager on the cluster, slurm.sh and generate_msa.py have been written. Make changes accordingly to make it work
# Run slurm.sh
